# Kings County Housing Prices Bakeoff

Below are a list of steps that you should take while trying to complete your bake-off entry.

## Step 1: Read in Data

In [30]:
import seaborn as sns
import scipy.stats as stats
from scipy import stats
from math import sqrt
import scipy
import numpy as np 
import datetime
import geopy.distance
import pandas as pd 
import sklearn
from sklearn.preprocessing import StandardScaler
from statsmodels.formula.api import ols
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn import metrics
import uszipcode
import pickle
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SelectKBest, f_regression,mutual_info_regression

In [ ]:
from uszipcode import SearchEngine
search = SearchEngine(simple_zipcode=True) # set simple_zipcode=False to use rich info database
zipcode = search.by_zipcode("10803")
#zipcode.to_dict()['median_household_income']
zipcode = zipcode.to_dict()
zipcode

In [ ]:
def zip_code_edu(df):
    education = []
    for zip in df['zipcode']:
        search = SearchEngine(simple_zipcode=True)
        zipcode = search.by_zipcode(zip)
        zipcode = zipcode.to_dict()
        income = zipcode['families_vs_singles']
        education.append(income)
    return education
zip_edu_attain = zip_code_edu(kc_df)

In [ ]:
def zip_code_mhi(df):
    median_income = []
    for zip in df['zipcode']:
        search = SearchEngine(simple_zipcode=True)
        zipcode = search.by_zipcode(zip)
        zipcode = zipcode.to_dict()
        income = zipcode['median_household_income']
        median_income.append(income)
    return median_income
m_house_income = zip_code_mhi(kc_df)

In [ ]:
def zip_code_pd(df):
    pop_density = []
    for zip in df['zipcode']:
        search = SearchEngine(simple_zipcode=True)
        zipcode = search.by_zipcode(zip)
        zipcode = zipcode.to_dict()
        income = zipcode['population_density']
        pop_density.append(income)
    return pop_density
p_density = zip_code_pd(kc_df)

In [ ]:
def zip_code(df):
    zip_code_zc = []
    for zip in df['zipcode']:
        search = SearchEngine(simple_zipcode=True)
        zipcode = search.by_zipcode(zip)
        zipcode = zipcode.to_dict()
        income = zipcode['zipcode']
        zip_code_zc.append(income)
    return zip_code_zc
l_zipcode = zip_code(kc_df)

In [ ]:
zip_pd = dict(zip(l_zipcode, p_density))
zip_pd

In [ ]:
zip_mhi = dict(zip(l_zipcode, m_house_income))
zip_mhi

## Step 2: Exploratory Data Analysis 
    
Become familiar with the data.  Look to see if there are any extreme values.  

Additionally create data visualizations to determine if there are any relationships between your features and your target variables.  

In [31]:
#reading data in
kc_df = pd.read_csv('kc_house_data_train.csv')
kc_house_test = pd.read_csv('kc_house_data_test_features.csv')
#zip_score = pd.read_csv('Niche.csv')[['zipcode']]
#kc_df = zip_score.set_index('zipcode').join(kc_house.set_index('zipcode'))

In [32]:
kc_df = kc_df.reset_index()

In [33]:
#Keeping the ID column
kc_id = kc_df['id']

#dropping the ID
kc_df.drop(['id', 'Unnamed: 0', 'date', 'sqft_living15', 'sqft_lot15' ], axis = 1, inplace = True)

target = kc_df['price']

In [34]:
kc_df.drop('index', axis = 1, inplace = True)

In [35]:
kc_df

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long
0,365000.0,4,2.25,2070,8893,2.0,0,0,4,8,2070,0,1986,0,98058,47.4388,-122.162
1,865000.0,5,3.00,2900,6730,1.0,0,0,5,8,1830,1070,1977,0,98115,47.6784,-122.285
2,1038000.0,4,2.50,3770,10893,2.0,0,2,3,11,3770,0,1997,0,98006,47.5646,-122.129
3,1490000.0,3,3.50,4560,14608,2.0,0,2,3,12,4560,0,1990,0,98034,47.6995,-122.228
4,711000.0,3,2.50,2550,5376,2.0,0,0,3,9,2550,0,2004,0,98052,47.6647,-122.083
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17285,750000.0,5,2.50,3240,9960,1.0,0,1,3,8,2020,1220,1958,0,98008,47.5858,-122.112
17286,440000.0,2,1.75,1300,4000,2.0,0,0,3,7,1300,0,1948,0,98105,47.6687,-122.288
17287,310000.0,3,1.00,1050,9876,1.0,0,0,3,7,1050,0,1953,0,98028,47.7635,-122.262
17288,427500.0,3,1.50,1900,43186,1.5,0,0,4,7,1300,600,1971,0,98038,47.4199,-121.990


In [ ]:
price_corr = kc_df[kc_df.columns[:]].corr()['price'][:]
price_corr.sort_values()

In [ ]:
#kc_house.corr()

def high_corr(df, alpha = 0.7):
    corr = df[[cor for cor in df.columns if cor != 'price']].corr()

    corr_tuples = [(f, list(s[abs(s) > alpha].index)) for f,s in corr.iteritems()]

    for t1, t2 in corr_tuples:
        if t1 in t2:
            t2.remove(t1)

    corr_f = [t for t in corr_tuples if len(t[1]) != 0]

    highly_corr = []
    for t in corr_f:
        for f in t[1]:
            highly_corr.append(f)

    return list(set(highly_corr))


In [ ]:
high_corr(kc_house)

In [ ]:
fig = plt.figure(figsize=(20,10))
plt.subplot(2,1,1)
plt.ylabel('Frequency')
sns.distplot(kc_house['price'])
plt.subplot(2,1,2)
plt.boxplot(kc_house['price'], vert= False)
plt.xlabel('price')
plt.show()

## Step 3: Clean up any issues (extreme values, etc.) with the data.  

Remember that you can't just delete rows with extreme values. Similar observations might be present in the holdout data set, and you can't just delete those rows and not have a prediction for it. 

Gameplan

- transform extreme values of rows, i.e. # of bedrooms (one with 33)
    
- using sqft living to estimate missing bathrooms

- turn basement into cat

- year built / reno

- check on grade vs housing built / reno

- view (as the view goes up it gets 'better and better')

Features

- Niche perhaps (school grades - cat dummy variable) [csv]

- Prox to Parks or Nature [geospat]

- effect of Population density [csv - census]

- relation between space and price [sqft vs price (some other feature)]

- population growth (increase of supply for limited demand)

- access to city center [geospat]

- prox to commerical businesses [geospat]

-squaring bd bath in our linear model



In [6]:
# kc_df['sqft_liv_lot'] = np.nan

# def n_sqft_liv_lot(row):
#     row['sqft_liv_lot'] = (row['sqft_living'] / row['sqft_lot'])    
#     return row

# kc_df = kc_df.apply(n_sqft_liv_lot, axis = 1)

In [7]:
# def n_sqft_living(row):
#     if row['sqft_living'] > 5000:
#         row['sqft_living'] = 5000    
#     return row

# kc_df = kc_df.apply(n_sqft_living, axis = 1)

In [8]:
# def n_sqft_lot(row):
#     if row['sqft_lot'] > 75000:
#         row['sqft_lot'] = 75000    
#     return row

# kc_df = kc_df.apply(n_sqft_lot, axis = 1)

In [36]:
def n_bed(row):
    if row['bedrooms'] ==0:
        row['bedrooms'] = row['floors']
    if row['bedrooms'] > 10:
        row['bedrooms'] = 10
    return row

kc_df = kc_df.apply(n_bed, axis = 1)

In [37]:
def n_bath(row):
    if row['bathrooms'] == 0:
        row['bathrooms'] = 1
    if row['bathrooms'] > 8:
        row['bathrooms'] = 8
    return row

kc_df = kc_df.apply(n_bath, axis = 1)

In [38]:
#Turn basements into a categorical set in the column

kc_df['sqft_basement'] = np.where(kc_df['sqft_basement'] != 0, 1, 0)

In [39]:
kc_df['yr_updated'] = np.nan

def yr_update(row):
    if row['yr_renovated'] == 0:
        row['yr_updated'] =2021 - row['yr_built']
    elif row['yr_renovated'] != 0:
        row['yr_updated'] = 2021 - row['yr_renovated']
    return row['yr_updated']

kc_df['yr_updated'] = kc_df.apply(yr_update, axis = 1)
kc_df.drop(['yr_built',	'yr_renovated' ], axis = 1, inplace = True)

In [40]:
def city_distance(row):
    dist = geopy.distance.geodesic((47.6205,122.3493), (row['lat'], abs(row['long'])))
    return dist.miles

kc_df['dist_city'] = kc_df.apply(city_distance, axis=1)
kc_df.drop(['lat',	'long' ], axis = 1, inplace = True)

In [14]:
# def school_grade(row):
#     if row['school_grade'] in ['A+', 'A', 'A-', 'A+ ', 'A ', 'A- ']:
#         row['school_grade'] = 5
#     if row['school_grade'] in ['B+', 'B', 'B-', 'B+ ', 'B ', 'B- ']:
#         row['school_grade'] = 3
#     if row['school_grade'] in ['C+', 'C', 'C-', 'C+ ', 'C ', 'C- ']:
#         row['school_grade'] = 1
#     return row

# kc_df = kc_df.apply(school_grade, axis=1)

In [41]:
kc_df

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,zipcode,yr_updated,dist_city
0,365000.0,4.0,2.25,2070.0,8893.0,2.0,0.0,0.0,4.0,8.0,2070.0,0,98058.0,35.0,15.309262
1,865000.0,5.0,3.00,2900.0,6730.0,1.0,0.0,0.0,5.0,8.0,1830.0,1,98115.0,44.0,5.001116
2,1038000.0,4.0,2.50,3770.0,10893.0,2.0,0.0,2.0,3.0,11.0,3770.0,0,98006.0,24.0,10.995967
3,1490000.0,3.0,3.50,4560.0,14608.0,2.0,0.0,2.0,3.0,12.0,4560.0,0,98034.0,31.0,7.863897
4,711000.0,3.0,2.50,2550.0,5376.0,2.0,0.0,0.0,3.0,9.0,2550.0,0,98052.0,17.0,12.802871
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17285,750000.0,5.0,2.50,3240.0,9960.0,1.0,0.0,1.0,3.0,8.0,2020.0,1,98008.0,63.0,11.343927
17286,440000.0,2.0,1.75,1300.0,4000.0,2.0,0.0,0.0,3.0,7.0,1300.0,0,98105.0,73.0,4.390826
17287,310000.0,3.0,1.00,1050.0,9876.0,1.0,0.0,0.0,3.0,7.0,1050.0,0,98028.0,68.0,10.685734
17288,427500.0,3.0,1.50,1900.0,43186.0,1.5,0.0,0.0,4.0,7.0,1300.0,1,98038.0,50.0,21.789572


In [13]:
no_p = list(kc_df.columns.drop('price'))

In [14]:
no_p

['bedrooms',
 'bathrooms',
 'sqft_living',
 'sqft_lot',
 'floors',
 'waterfront',
 'view',
 'condition',
 'grade',
 'sqft_basement',
 'zipcode',
 'yr_updated',
 'dist_city']

# Setting up our Dummies

In [42]:
cat_var = ['zipcode', 'condition', 'view', 'floors', 'grade', 'waterfront','sqft_basement']
cont_var = ['bedrooms', 'bathrooms', 'dist_city', 'sqft_living', 'sqft_above', 'sqft_lot', 'yr_updated']

In [43]:
# your code here
# fuction to generate a dummy variable for a feature
# needs tweeks
def to_dummies(df, features):
   
    d_df = []
    for f in features:
        d_df.append(pd.get_dummies(df[f], prefix='{}'.format(str(f)[:3]), drop_first=True))
    df = df.drop(features, axis = 1)
    df = pd.concat([df] + d_df ,axis=1)
    return df


In [44]:
kc_df = to_dummies(kc_df, cat_var)

In [45]:
kc_df

,price,bedrooms,bathrooms,sqft_living,sqft_lot,sqft_above,yr_updated,dist_city,zip_98002.0,zip_98003.0,...,gra_6.0,gra_7.0,gra_8.0,gra_9.0,gra_10.0,gra_11.0,gra_12.0,gra_13.0,wat_1.0,sqf_1
0,365000.0,4.0,2.25,2070.0,8893.0,2070.0,35.0,15.309262,0,0,...,0,0,1,0,0,0,0,0,0,0
1,865000.0,5.0,3.00,2900.0,6730.0,1830.0,44.0,5.001116,0,0,...,0,0,1,0,0,0,0,0,0,1
2,1038000.0,4.0,2.50,3770.0,10893.0,3770.0,24.0,10.995967,0,0,...,0,0,0,0,0,1,0,0,0,0
3,1490000.0,3.0,3.50,4560.0,14608.0,4560.0,31.0,7.863897,0,0,...,0,0,0,0,0,0,1,0,0,0
4,711000.0,3.0,2.50,2550.0,5376.0,2550.0,17.0,12.802871,0,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17285,750000.0,5.0,2.50,3240.0,9960.0,2020.0,63.0,11.343927,0,0,...,0,0,1,0,0,0,0,0,0,1
17286,440000.0,2.0,1.75,1300.0,4000.0,1300.0,73.0,4.390826,0,0,...,0,1,0,0,0,0,0,0,0,0
17287,310000.0,3.0,1.00,1050.0,9876.0,1050.0,68.0,10.685734,0,0,...,0,1,0,0,0,0,0,0,0,0
17288,427500.0,3.0,1.50,1900.0,43186.0,1300.0,50.0,21.789572,0,0,...,0,1,0,0,0,0,0,0,0,1


In [46]:
summary_features = list(kc_df.columns.drop('price'))

In [119]:
lol = list(kc_df.columns.drop('price'))

In [ ]:
not_dummies = ['population',
 'bedrooms',
 'bathrooms',
 'sqft_living',
 'sqft_lot',
 'sqft_above',
 'sqft_basement',
 'yr_built',
 'yr_renovated',
 'lat',
 'long',
 'sqft_living15',
 'sqft_lot15',
 'sqft_liv_lot',
 'yr_updated',
 'dist_city']



In [ ]:
fix, ax = plt.subplots(4, 2, figsize=(50,75))
for variables, subplot in zip(cat_data, ax.flatten()):
    sns.countplot(kc_house, ax=subplot)
    for label in subplot.get_xticklabels():
        lebel.set_rotation(90) 

In [ ]:
fig, ax = plt.subplots(3, 3, figsize = (60, 40))
for var, subplot in zip(cat_data, ax.flatten()):
    sns.boxplot(x=var, y='price', data = kc_house, ax=subplot)

## Step 4: Generate new features that you think could be important.

After doing this, you will want to go back to steps 2 and 3 to investigate these new features.

### 4.1) Identify a categorical variable in the data set and create dummy columns.

### 4.2) There is a column that gives the date for when the house was sold, how could this be useful in your model? How might you transform the current column to a more useful feature?

### 4.3) There are columns for when the house was built and when it was renovated.  How could you use these columns to create a new column?

### <ins>Non-linear transformations</ins>

### 4.4) Create a polynomial feature for two of your continuous variables.

In [ ]:
# your code here

In [47]:
poly = PolynomialFeatures(degree = 2, include_bias = False)

In [48]:
poly_data = poly.fit_transform(kc_df[cont_var])

In [49]:
poly_columns = poly.get_feature_names(kc_df[cont_var].columns)

In [50]:
df_poly = pd.DataFrame(poly_data, columns = poly_columns) 

In [54]:
df_poly

,bedrooms,bathrooms,dist_city,sqft_living,sqft_above,sqft_lot,yr_updated,bedrooms^2,bedrooms bathrooms,bedrooms dist_city,...,sqft_living^2,sqft_living sqft_above,sqft_living sqft_lot,sqft_living yr_updated,sqft_above^2,sqft_above sqft_lot,sqft_above yr_updated,sqft_lot^2,sqft_lot yr_updated,yr_updated^2
0,4.0,2.25,15.309262,2070.0,2070.0,8893.0,35.0,16.0,9.0,61.237048,...,4284900.0,4284900.0,18408510.0,72450.0,4284900.0,18408510.0,72450.0,7.908545e+07,311255.0,1225.0
1,5.0,3.00,5.001116,2900.0,1830.0,6730.0,44.0,25.0,15.0,25.005578,...,8410000.0,5307000.0,19517000.0,127600.0,3348900.0,12315900.0,80520.0,4.529290e+07,296120.0,1936.0
2,4.0,2.50,10.995967,3770.0,3770.0,10893.0,24.0,16.0,10.0,43.983866,...,14212900.0,14212900.0,41066610.0,90480.0,14212900.0,41066610.0,90480.0,1.186574e+08,261432.0,576.0
3,3.0,3.50,7.863897,4560.0,4560.0,14608.0,31.0,9.0,10.5,23.591691,...,20793600.0,20793600.0,66612480.0,141360.0,20793600.0,66612480.0,141360.0,2.133937e+08,452848.0,961.0
4,3.0,2.50,12.802871,2550.0,2550.0,5376.0,17.0,9.0,7.5,38.408613,...,6502500.0,6502500.0,13708800.0,43350.0,6502500.0,13708800.0,43350.0,2.890138e+07,91392.0,289.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17285,5.0,2.50,11.343927,3240.0,2020.0,9960.0,63.0,25.0,12.5,56.719634,...,10497600.0,6544800.0,32270400.0,204120.0,4080400.0,20119200.0,127260.0,9.920160e+07,627480.0,3969.0
17286,2.0,1.75,4.390826,1300.0,1300.0,4000.0,73.0,4.0,3.5,8.781652,...,1690000.0,1690000.0,5200000.0,94900.0,1690000.0,5200000.0,94900.0,1.600000e+07,292000.0,5329.0
17287,3.0,1.00,10.685734,1050.0,1050.0,9876.0,68.0,9.0,3.0,32.057202,...,1102500.0,1102500.0,10369800.0,71400.0,1102500.0,10369800.0,71400.0,9.753538e+07,671568.0,4624.0
17288,3.0,1.50,21.789572,1900.0,1300.0,43186.0,50.0,9.0,4.5,65.368716,...,3610000.0,2470000.0,82053400.0,95000.0,1690000.0,56141800.0,65000.0,1.865031e+09,2159300.0,2500.0


In [55]:
#
kc_df_poly2 = pd.merge(df_poly, kc_df, left_index=True, right_index=True)


In [56]:
kc_df_poly2

,bedrooms_x,bathrooms_x,dist_city_x,sqft_living_x,sqft_above_x,sqft_lot_x,yr_updated_x,bedrooms^2,bedrooms bathrooms,bedrooms dist_city,...,gra_6.0,gra_7.0,gra_8.0,gra_9.0,gra_10.0,gra_11.0,gra_12.0,gra_13.0,wat_1.0,sqf_1
0,4.0,2.25,15.309262,2070.0,2070.0,8893.0,35.0,16.0,9.0,61.237048,...,0,0,1,0,0,0,0,0,0,0
1,5.0,3.00,5.001116,2900.0,1830.0,6730.0,44.0,25.0,15.0,25.005578,...,0,0,1,0,0,0,0,0,0,1
2,4.0,2.50,10.995967,3770.0,3770.0,10893.0,24.0,16.0,10.0,43.983866,...,0,0,0,0,0,1,0,0,0,0
3,3.0,3.50,7.863897,4560.0,4560.0,14608.0,31.0,9.0,10.5,23.591691,...,0,0,0,0,0,0,1,0,0,0
4,3.0,2.50,12.802871,2550.0,2550.0,5376.0,17.0,9.0,7.5,38.408613,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17285,5.0,2.50,11.343927,3240.0,2020.0,9960.0,63.0,25.0,12.5,56.719634,...,0,0,1,0,0,0,0,0,0,1
17286,2.0,1.75,4.390826,1300.0,1300.0,4000.0,73.0,4.0,3.5,8.781652,...,0,1,0,0,0,0,0,0,0,0
17287,3.0,1.00,10.685734,1050.0,1050.0,9876.0,68.0,9.0,3.0,32.057202,...,0,1,0,0,0,0,0,0,0,0
17288,3.0,1.50,21.789572,1900.0,1300.0,43186.0,50.0,9.0,4.5,65.368716,...,0,1,0,0,0,0,0,0,0,1


In [57]:
summary_features1 = list(kc_df_poly2.columns.drop('price'))

In [36]:
summary_features1

['bedrooms_x',
 'bathrooms_x',
 'dist_city_x',
 'sqft_living_x',
 'sqft_lot_x',
 'yr_updated_x',
 'bedrooms^2',
 'bedrooms bathrooms',
 'bedrooms dist_city',
 'bedrooms sqft_living',
 'bedrooms sqft_lot',
 'bedrooms yr_updated',
 'bathrooms^2',
 'bathrooms dist_city',
 'bathrooms sqft_living',
 'bathrooms sqft_lot',
 'bathrooms yr_updated',
 'dist_city^2',
 'dist_city sqft_living',
 'dist_city sqft_lot',
 'dist_city yr_updated',
 'sqft_living^2',
 'sqft_living sqft_lot',
 'sqft_living yr_updated',
 'sqft_lot^2',
 'sqft_lot yr_updated',
 'yr_updated^2',
 'bedrooms_y',
 'bathrooms_y',
 'sqft_living_y',
 'sqft_lot_y',
 'yr_updated_y',
 'dist_city_y',
 'zip_98002.0',
 'zip_98003.0',
 'zip_98004.0',
 'zip_98005.0',
 'zip_98006.0',
 'zip_98007.0',
 'zip_98008.0',
 'zip_98010.0',
 'zip_98011.0',
 'zip_98014.0',
 'zip_98019.0',
 'zip_98022.0',
 'zip_98023.0',
 'zip_98024.0',
 'zip_98027.0',
 'zip_98028.0',
 'zip_98029.0',
 'zip_98030.0',
 'zip_98031.0',
 'zip_98032.0',
 'zip_98033.0',
 'zip_98

In [86]:
(kc_df_poly2.columns.drop('price'), inplace = True)

SyntaxError: invalid syntax (<ipython-input-86-3c567e99f885>, line 1)

### 4.5) Create an interaction feature between a binary variable (dummy variable) and a continuous variable.

## Step 5: Train-Test Split

If you plan on doing any scaling of your data, make sure it is done at the appropriate time. 

In [40]:
def f_test_selection(df, target, k):
    """
    k - Select features according to the k highest scores
    """
    X_train, X_test, y_train, y_test = train_test_split(df, target, random_state=9, test_size=0.2)
    scaler = StandardScaler()
    # fit the scaler to the training data
    scaler.fit(X_train)
    #transform the training data
    scaled_data = scaler.transform(X_train)
    X_train_scaled = pd.DataFrame(data=scaled_data, columns=df.columns, index=X_train.index)
    #transform the test data
    scaled_test_data = scaler.transform(X_test)
    #create dataframe
    X_test_scaled = pd.DataFrame(data=scaled_test_data, columns=df.columns, index=X_test.index)
    selector = SelectKBest(f_regression, k= k)
    selector.fit(X_train_scaled, y_train)
    selected_columns = X_train_scaled.columns[selector.get_support()]
    removed_columns = X_train_scaled.columns[~selector.get_support()]
    return selected_columns

In [55]:
best_f = f_test_selection(kc_df_poly2, target, 120)

In [56]:
best_f

Index(['bedrooms_x', 'bathrooms_x', 'dist_city_x', 'sqft_living_x',
       'sqft_lot_x', 'yr_updated_x', 'bedrooms^2', 'bedrooms bathrooms',
       'bedrooms dist_city', 'bedrooms sqft_living', 'bedrooms sqft_lot',
       'bathrooms^2', 'bathrooms sqft_living', 'bathrooms sqft_lot',
       'bathrooms yr_updated', 'dist_city^2', 'dist_city sqft_living',
       'dist_city yr_updated', 'sqft_living^2', 'sqft_living sqft_lot',
       'sqft_living yr_updated', 'price', 'bedrooms_y', 'bathrooms_y',
       'sqft_living_y', 'sqft_lot_y', 'yr_updated_y', 'dist_city_y',
       'zip_98002.0', 'zip_98003.0', 'zip_98004.0', 'zip_98006.0',
       'zip_98022.0', 'zip_98023.0', 'zip_98030.0', 'zip_98031.0',
       'zip_98033.0', 'zip_98038.0', 'zip_98039.0', 'zip_98040.0',
       'zip_98042.0', 'zip_98055.0', 'zip_98058.0', 'zip_98075.0',
       'zip_98092.0', 'zip_98102.0', 'zip_98105.0', 'zip_98106.0',
       'zip_98112.0', 'zip_98119.0', 'zip_98168.0', 'zip_98178.0',
       'zip_98198.0', 'zip_9819

In [59]:
eff_feat2 = ['bedrooms_x', 'bathrooms_x', 'dist_city_x', 'sqft_living_x',
       'sqft_lot_x', 'yr_updated_x', 'bedrooms^2', 'bedrooms bathrooms',
       'bedrooms dist_city', 'bedrooms sqft_living', 'bedrooms sqft_lot',
       'bathrooms^2', 'bathrooms sqft_living', 'bathrooms sqft_lot',
       'bathrooms yr_updated', 'dist_city^2', 'dist_city sqft_living',
       'dist_city yr_updated', 'sqft_living^2', 'sqft_living sqft_lot',
       'sqft_living yr_updated', 'bedrooms_y', 'bathrooms_y',
       'sqft_living_y', 'sqft_lot_y', 'yr_updated_y', 'dist_city_y',
       'zip_98002.0', 'zip_98003.0', 'zip_98004.0', 'zip_98006.0',
       'zip_98022.0', 'zip_98023.0', 'zip_98030.0', 'zip_98031.0',
       'zip_98033.0', 'zip_98038.0', 'zip_98039.0', 'zip_98040.0',
       'zip_98042.0', 'zip_98055.0', 'zip_98058.0', 'zip_98075.0',
       'zip_98092.0', 'zip_98102.0', 'zip_98105.0', 'zip_98106.0',
       'zip_98112.0', 'zip_98119.0', 'zip_98168.0', 'zip_98178.0',
       'zip_98198.0', 'zip_98199.0', 'vie_1.0', 'vie_2.0', 'vie_3.0',
       'vie_4.0', 'flo_2.0', 'flo_2.5', 'gra_5.0', 'gra_6.0', 'gra_7.0',
       'gra_9.0', 'gra_10.0', 'gra_11.0', 'gra_12.0', 'gra_13.0', 'wat_1.0',
       'sqf_1']

In [52]:
eff_feat = ['bedrooms_x', 'bathrooms_x', 'dist_city_x', 'sqft_living_x',
       'sqft_lot_x', 'yr_updated_x', 'bedrooms^2', 'bedrooms bathrooms',
       'bedrooms dist_city', 'bedrooms sqft_living', 'bedrooms sqft_lot',
       'bedrooms yr_updated', 'bathrooms^2', 'bathrooms sqft_living',
       'bathrooms sqft_lot', 'bathrooms yr_updated', 'dist_city^2',
       'dist_city sqft_living', 'dist_city sqft_lot', 'dist_city yr_updated',
       'sqft_living^2', 'sqft_living sqft_lot', 'sqft_living yr_updated',
       'sqft_lot^2', 'yr_updated^2', 'bedrooms_y', 'bathrooms_y',
       'sqft_living_y', 'sqft_lot_y', 'yr_updated_y', 'dist_city_y',
       'zip_98002.0', 'zip_98003.0', 'zip_98004.0', 'zip_98005.0',
       'zip_98006.0', 'zip_98008.0', 'zip_98019.0', 'zip_98022.0',
       'zip_98023.0', 'zip_98027.0', 'zip_98030.0', 'zip_98031.0',
       'zip_98032.0', 'zip_98033.0', 'zip_98038.0', 'zip_98039.0',
       'zip_98040.0', 'zip_98042.0', 'zip_98045.0', 'zip_98052.0',
       'zip_98053.0', 'zip_98055.0', 'zip_98056.0', 'zip_98058.0',
       'zip_98074.0', 'zip_98075.0', 'zip_98077.0', 'zip_98092.0',
       'zip_98102.0', 'zip_98105.0', 'zip_98106.0', 'zip_98108.0',
       'zip_98109.0', 'zip_98112.0', 'zip_98115.0', 'zip_98118.0',
       'zip_98119.0', 'zip_98126.0', 'zip_98133.0', 'zip_98146.0',
       'zip_98148.0', 'zip_98155.0', 'zip_98168.0', 'zip_98177.0',
       'zip_98178.0', 'zip_98188.0', 'zip_98198.0', 'zip_98199.0', 'con_2.0',
       'con_4.0', 'con_5.0', 'vie_1.0', 'vie_2.0', 'vie_3.0', 'vie_4.0',
       'flo_2.0', 'flo_2.5', 'gra_4.0', 'gra_5.0', 'gra_6.0', 'gra_7.0',
       'gra_9.0', 'gra_10.0', 'gra_11.0', 'gra_12.0', 'gra_13.0', 'wat_1.0',
       'sqf_1']

In [23]:
model_test(kc_df, summary_features, 'price')

NameError: name 'model_test' is not defined

In [60]:
model_test(kc_df_poly2, eff_feat2, 'price')

Training:  142930 vs. Testing:  163799


### 5.1) Perform a train-test split of the data.

In [48]:
target = kc_df['price']

In [58]:
def model_test(df, features, target):
    target = df[target]
    df_feature = df[features]
    x_train, x_test, y_train, y_test = train_test_split(df_feature, 
    target, random_state = 34, test_size = 0.2)
    lm = linear_model.LinearRegression()
    lm = lm.fit(x_train, y_train)

    y_train_pred = lm.predict(x_train)
    train_rmse = np.sqrt(metrics.mean_squared_error(y_train, y_train_pred))

    y_test_pred = lm.predict(x_test)
    test_rmse = np.sqrt(metrics.mean_squared_error(y_test, y_test_pred))

    print('Training: ', int(train_rmse), 'vs. Testing: ', int(test_rmse))


Model Test with Dummy Variables

In [25]:
model_test(kc_df, summary_features, 'price')

Training:  150152 vs. Testing:  156737


Model Test with Dummy Variables & Polynomials 

In [59]:
model_test(kc_df_poly2, summary_features1, 'price')

Training:  130248 vs. Testing:  150555


Model Test with Dummy Variables & Polynomials with the 'best' 100 features

In [100]:
model_test(kc_df_poly2, best_features, 'price')

Training:  143934 vs. Testing:  139763


## kc_df Train Test Split

In [46]:
#your code here 
#improt train_test_split from sklearn package
from sklearn.model_selection import train_test_split
#call train_test_split on the data and capture the results
X_train, X_test, y_train, y_test = train_test_split(kc_df_poly2[summary_features1], kc_df['price'], random_state=34,test_size=0.2)
#showing number of summary features
#check the shape of the results
print("Training set - Features: ", X_train.shape, "Target: ", y_train.shape)
print("Training set - Features: ", X_test.shape, "Target: ",y_test.shape)

Training set - Features:  (13832, 128) Target:  (13832,)
Training set - Features:  (3458, 128) Target:  (3458,)


In [49]:
from sklearn import linear_model

#instantiate a linear regression object
lm = linear_model.LinearRegression()

#fit the linear regression to the data
lm = lm.fit(X_train, y_train)


#print the y intercept of our model
print(lm.intercept_)
#coefs for each of the summary features
print(lm.coef_)

-23710.128205479356
[ 1.69655366e+04 -2.25406327e+04 -2.09018788e+03  1.28646484e+02
  9.59046608e-01 -3.89261997e+02 -3.00048246e+03  3.25335172e+03
  1.91363625e+03 -1.88498191e+01  1.48659822e-01  1.25834214e+02
  1.92992783e+02  7.00944813e+02  1.35443667e+01  1.80885150e-02
  1.27429999e+02  2.57952405e+02 -1.25812593e+01 -1.24411675e-02
 -6.31478885e+01  1.96977976e-02 -3.85625920e-04 -4.84306103e-01
 -2.28119461e-07 -1.05695322e-02  1.19429316e+01  2.16846915e+03
 -2.25408169e+04  1.28652021e+02  9.59021120e-01 -3.89261861e+02
 -2.09019150e+03 -2.20824454e+03 -1.77345141e+03  5.06400026e+05
  1.01994825e+05  9.34813571e+04  9.57601268e+04  1.21229546e+05
  1.16159305e+05  9.62944915e+03  7.07794144e+04  7.36072801e+04
  6.69997773e+04 -1.01456191e+04  1.63004856e+05  1.13005148e+05
 -1.77565495e+04  1.61538063e+05 -2.48345542e+04 -4.36228573e+04
 -4.57341675e+04  1.53228605e+05  3.78112221e+04  5.79640881e+04
  7.90253489e+05  2.52074457e+05 -5.40588000e+03  1.80937856e+05
  1.0

In [50]:
print ("R^2 Score:", lm.score(X_train, y_train))

R^2 Score: 0.8676553542263472


In [51]:
y_train_pred = lm.predict(X_train)

In [52]:
from sklearn import metrics


train_rmse = np.sqrt(metrics.mean_squared_error(y_train, y_train_pred))


print('Root Mean Squared Error:' , train_rmse)

Root Mean Squared Error: 135797.66590476874


In [53]:
price_std = target.std()


print('Z-Score of Root Mean Squared Error:' , train_rmse/price_std)

Z-Score of Root Mean Squared Error: 0.36375773436359166


In [54]:
y_pred = lm.predict(X_test)

In [55]:
print ("Score:", lm.score(X_test, y_test))

Score: 0.8765857644799833


In [56]:
price_std = target.std()
test_mae = metrics.mean_absolute_error(y_test, y_pred)
test_rmse = np.sqrt(metrics.mean_squared_error(y_test, y_pred))


print('Mean Absolute Error:' + str(metrics.mean_absolute_error(y_test, y_pred)))
print('Mean Squared Error:' + str(metrics.mean_squared_error(y_test, y_pred)))
print('Root Mean Squared Error:' + str(np.sqrt(metrics.mean_squared_error(y_test, y_pred))))

Mean Absolute Error:79952.7721028022
Mean Squared Error:17207682382.762886
Root Mean Squared Error:131178.0560260095


In [57]:
print('Mean Absolute Error  Z:', test_mae/price_std )
print('Root Mean Squared Error Z:' , test_rmse/price_std)

Mean Absolute Error  Z: 0.21416744568054166
Root Mean Squared Error Z: 0.35138330353707387


In [58]:
print('Training: ', int(train_rmse), "vs. Testing: ", int(test_rmse))

Training:  135797 vs. Testing:  131178


### 5.2) Fit your scaler to training the data.

In [59]:
#your code here 

In [60]:
# ASK SEAN ABOUT NUMBER
X_train.select_dtypes(include=['number']).columns

Index(['bedrooms_x', 'bathrooms_x', 'dist_city_x', 'sqft_living_x',
       'sqft_lot_x', 'yr_updated_x', 'bedrooms^2', 'bedrooms bathrooms',
       'bedrooms dist_city', 'bedrooms sqft_living',
       ...
       'gra_6.0', 'gra_7.0', 'gra_8.0', 'gra_9.0', 'gra_10.0', 'gra_11.0',
       'gra_12.0', 'gra_13.0', 'wat_1.0', 'sqf_1'],
      dtype='object', length=128)

In [61]:
#Features we want to scale (continuous). We do not want to scale dummy (why?) and categorical
 
scaled_features = cont_var

In [62]:
#Separating our non-scaled data

not_scaled = [x for x in X_train.columns if x not in cont_var]

In [63]:
scaler = StandardScaler()

scaler.fit(kc_df_poly2)

scaled_data = scaler.transform(kc_df_poly2)

In [57]:
# setting scaler equal to our function
# scaler = StandardScaler()
# # fit the scaler to the training data
# scaler.fit(X_train[s_features])
# #transform the training data
# scaled_data = scaler.transform(X_train)

In [64]:
# #create dataframe
kc_df_poly2_scaled = pd.DataFrame(data=scaled_data, columns=kc_df_poly2.columns, index=kc_df_poly2.index)

In [ ]:
# X_train = pd.concat([X_train_scaled, X_train[not_scaled]], axis=1)

In [81]:
kc_df_poly2['price']

0         309000.0
1         286651.0
2         260000.0
3         165000.0
4         185900.0
           ...    
17285     510000.0
17286    1115000.0
17287     756000.0
17288     875000.0
17289     601000.0
Name: price, Length: 17290, dtype: float64

In [79]:
model_test(kc_df_poly2_scaled, summary_features1, 'price')

Training:  0 vs. Testing:  0


## kc_df_poly2

In [81]:
#improt train_test_split from sklearn package
from sklearn.model_selection import train_test_split
#call train_test_split on the data and capture the results
X_train, X_test, y_train, y_test = train_test_split(kc_df_poly2[summary_features1], kc_df_poly2['price'], random_state=24,test_size=0.2)
#showing number of summary features
#check the shape of the results
print("Training set - Features: ", X_train.shape, "Target: ", y_train.shape)
print("Training set - Features: ", X_test.shape, "Target: ",y_test.shape)

Training set - Features:  (13832, 121) Target:  (13832,)
Training set - Features:  (3458, 121) Target:  (3458,)


In [88]:
#instantiate a linear regression object
lm = linear_model.LinearRegression()

#fit the linear regression to the data
lm = lm.fit(X_train, y_train)


#print the y intercept of our model
print(lm.intercept_)
#coefs for each of the summary features
print(lm.coef_)

7570591.159768466
[-4.23598869e+03 -8.84218124e+00 -2.40215229e-01  3.70141505e+02
 -1.08109612e+01 -2.32129195e-02  3.17644632e-02 -1.88318468e-04
  2.90176690e-05 -2.50984571e+00 -7.02143932e+03  2.91935778e+04
 -8.84238273e+00 -2.40194970e-01  1.55344336e+02  6.59790511e+04
 -3.57730744e+02  6.05764833e+00 -1.71904627e+05 -1.06291103e+04
  4.24556712e+01  1.66687330e-01 -9.69662670e+04 -5.60110577e+02
 -4.30205065e+03  5.19017750e+03  3.00042737e+04  3.93999306e+05
 -6.21942818e+04 -1.71706579e+04 -1.77512812e+04  1.20105706e+04
  4.97928979e+04 -7.26020839e+04 -7.11172647e+04 -2.08267393e+04
  2.15662709e+04  1.92183568e+04  2.87653255e+04  1.61615786e+04
 -1.23333611e+05  8.51839566e+04 -7.11819065e+03 -3.04207846e+04
 -3.19184209e+04  7.09695820e+04 -2.83308668e+04  8.82994848e+04
  6.78389448e+05  8.63270897e+04  4.22298096e+04  1.77056467e+05
  1.00309281e+05  2.52663244e+04 -8.22765912e+04 -4.40663334e+04
 -1.20505391e+05 -1.33680073e+05  6.63587156e+04 -2.84597066e+05
 -4.612

In [83]:
print ("R^2 Score:", lm.score(X_train, y_train))

R^2 Score: 0.8589150426170823


In [84]:
y_train_pred = lm.predict(X_train)

In [85]:
train_rmse = np.sqrt(metrics.mean_squared_error(y_train, y_train_pred))


print('Root Mean Squared Error:' , train_rmse)

Root Mean Squared Error: 142360.20237314573


In [87]:
price2_std = kc_df_poly2['price'].std()


print('Z-Score of Root Mean Squared Error:' , train_rmse/price2_std)

Z-Score of Root Mean Squared Error: 0.38133663295150494


In [89]:
y_pred = lm.predict(X_test)

In [90]:
print ("Score:", lm.score(X_test, y_test))

Score: 0.8573374646928246


In [91]:
test_mae = metrics.mean_absolute_error(y_test, y_pred)
test_rmse = np.sqrt(metrics.mean_squared_error(y_test, y_pred))


print('Mean Absolute Error:' + str(metrics.mean_absolute_error(y_test, y_pred)))
print('Mean Squared Error:' + str(metrics.mean_squared_error(y_test, y_pred)))
print('Root Mean Squared Error:' + str(np.sqrt(metrics.mean_squared_error(y_test, y_pred))))

Mean Absolute Error:80085.07657980137
Mean Squared Error:17425169148.12298
Root Mean Squared Error:132004.42851708794


In [92]:
print('Mean Absolute Error  Z:', test_mae/price2_std )
print('Root Mean Squared Error Z:' , test_rmse/price2_std)

Mean Absolute Error  Z: 0.21452184629912907
Root Mean Squared Error Z: 0.35359688639280495


In [93]:
print('Training: ', int(train_rmse), "vs. Testing: ", int(test_rmse))

Training:  142360 vs. Testing:  132004


### 5.3) Transform the testing set with the scaler.

In [ ]:
#your code here 

In [68]:
scaled_test_data = scaler.transform(X_test)

In [69]:
# Create the dataframe
X_test_scaled = pd.DataFrame(data=scaled_test_data, columns=X_test.columns, index=X_test.index)

In [ ]:
#X_test = pd.concat([X_test_scaled, X_test[not_scaled]], axis=1)

In [123]:
X_test_scaled

,population,bedrooms,bathrooms,sqft_living,sqft_lot,sqft_above,sqft_basement,yr_built,yr_renovated,lat,...,gra_5,gra_6,gra_7,gra_8,gra_9,gra_10,gra_11,gra_12,gra_13,wat_1
2314,-1.048833,-0.410471,-0.797278,-0.298924,4.430921,0.028016,-0.809284,-0.061749,-0.208633,-2.479485,...,-0.106108,-0.325078,1.179762,-0.619702,-0.371062,-0.234504,-0.136773,-0.064891,-0.024056,-0.091563
10893,-0.645017,0.694772,-1.446286,-0.700836,-0.547806,-0.540082,1.235659,-1.456684,-0.208633,-0.195835,...,-0.106108,-0.325078,-0.847629,1.613679,-0.371062,-0.234504,-0.136773,-0.064891,-0.024056,-0.091563
8474,-1.511105,0.694772,0.825243,1.366142,-0.443208,1.780656,-0.809284,1.095027,-0.208633,-0.135034,...,-0.106108,-0.325078,-0.847629,-0.619702,2.694966,-0.234504,-0.136773,-0.064891,-0.024056,-0.091563
4544,0.407041,-0.410471,0.500739,0.746049,-0.309225,1.127948,-0.809284,0.652730,-0.208633,0.936218,...,-0.106108,-0.325078,-0.847629,-0.619702,2.694966,-0.234504,-0.136773,-0.064891,-0.024056,-0.091563
13527,0.369945,-1.515715,-0.797278,-1.274997,-0.689976,-0.999394,-0.809284,0.754799,-0.208633,0.421583,...,-0.106108,-0.325078,1.179762,-0.619702,-0.371062,-0.234504,-0.136773,-0.064891,-0.024056,-0.091563
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5617,-0.700375,0.694772,-0.472774,0.493418,4.430921,-0.346687,1.235659,-0.027726,-0.208633,-0.184978,...,-0.106108,-0.325078,-0.847629,1.613679,-0.371062,-0.234504,-0.136773,-0.064891,-0.024056,-0.091563
982,-1.248825,-0.410471,-0.472774,-0.356340,-0.092402,-0.032420,-0.809284,-0.470022,-0.208633,0.540289,...,-0.106108,-0.325078,-0.847629,1.613679,-0.371062,-0.234504,-0.136773,-0.064891,-0.024056,-0.091563
5553,-0.700375,-1.515715,-1.446286,-1.435762,-0.557601,-1.168615,-0.809284,-0.095772,-0.208633,0.202266,...,-0.106108,-0.325078,1.179762,-0.619702,-0.371062,-0.234504,-0.136773,-0.064891,-0.024056,-0.091563
4418,0.407041,0.694772,0.500739,0.814948,-0.338120,1.200471,-0.809284,1.129050,-0.208633,0.766845,...,-0.106108,-0.325078,-0.847629,-0.619702,2.694966,-0.234504,-0.136773,-0.064891,-0.024056,-0.091563


In [ ]:
X_train_scaled 
X_teset_scaled

### 5.4) Fit the model to the training data.

In [ ]:
#your code here 

In [66]:
#instantiate a linear regression object
lm = LinearRegression()

In [67]:
#fit the linear regression to the data
lm = lm.fit(X_train, y_train)

In [68]:
y_train_pred = lm.predict(X_train)

In [69]:
train_rmse = np.sqrt(metrics.mean_squared_error(y_train, y_train_pred))


print('Training Root Mean Squared Error:' , train_rmse)

Training Root Mean Squared Error: 135797.66590476874


In [75]:
# Now we want to evaluate on the test set

In [70]:
y_test_pred = lm.predict(X_test)

#evaluate the predictions on the test examples
test_rmse = np.sqrt(metrics.mean_squared_error(y_test, y_test_pred))

print('Testing Root Mean Squared Error:' , test_rmse)


print('Training: ', int(train_rmse), "vs. Testing: ", int(test_rmse))

Testing Root Mean Squared Error: 131178.0560260095
Training:  135797 vs. Testing:  131178


In [ ]:
sns.residplot( y_test, y_test_pred,lowess=True, color="g")

## Log test start

In [ ]:
# Log test

y_log = np.log(y_train)

In [ ]:
#instantiate a linear regression object
lm_log = LinearRegression()

#fit the linear regression to the data
lm_log = lm_log.fit(X_train, y_log)

log_train_pred = lm_log.predict(X_train)

In [ ]:
#log_train_pred the predictions to get them on the same original scale 
y_train_pred = np.exp(log_train_pred)

In [ ]:
train_rmse = np.sqrt(metrics.mean_squared_error(y_train, log_train_pred))


print('Training Root Mean Squared Error:' , train_rmse)

In [60]:
print('Perfomance : {} %'.format(round(abs((test_rmse-train_rmse)/train_rmse)*100)))


NameError: name 'test_rmse' is not defined

### 5.5) Use the model to predict on the training set and the test set.

In [ ]:
#your code here 

In [ ]:
y_train_pred = lm.predict(X_train)
y_test_pred = lm.predict(X_test)

In [ ]:
y_train_pred

### 5.6) Evaluate the training and test predictions using RMSE.

In [ ]:
#your code here 

In [ ]:
rmse = np.sqrt(metrics.mean_squared_error(y_train, y_train_pred))
test_rmse = np.sqrt(metrics.mean_squared_error(y_test, y_test_pred))
print(‘Training Root Mean Squared Error:’ , train_rmse)
print(‘Testing Root Mean Squared Error:’ , test_rmse)

### 5.7) Determine if your model is overfit.

In [ ]:
#your code here 

In [72]:
print('Perfomance : {} %'.format(round(abs((test_rmse-train_rmse)/train_rmse)*100)))


Perfomance : 3 %


## Step 6: Utilize some different feature selection techniques before or in conjuction with fitting your models.

### 6.1) Utilize a filter method to identify some features to remove from the model.  

In [ ]:
# your code here

In [95]:
def f_test_selection(df, target, k):
    """
    k - Select features according to the k highest scores
    """
    X_train, X_test, y_train, y_test = train_test_split(df, target, random_state=9, test_size=0.2)
    scaler = StandardScaler()
    # fit the scaler to the training data
    scaler.fit(X_train)
    #transform the training data
    scaled_data = scaler.transform(X_train)
    X_train_scaled = pd.DataFrame(data=scaled_data, columns=df.columns, index=X_train.index)
    #transform the test data
    scaled_test_data = scaler.transform(X_test)
    #create dataframe
    X_test_scaled = pd.DataFrame(data=scaled_test_data, columns=df.columns, index=X_test.index)
    selector = SelectKBest(f_regression, k= k)
    selector.fit(X_train_scaled, y_train)
    selected_columns = X_train_scaled.columns[selector.get_support()]
    removed_columns = X_train_scaled.columns[~selector.get_support()]
    return selected_columns

In [96]:
f_test_selection(kc_df_poly2, kc_df['price'], 100)

Index(['dist_city_x', 'sqft_living_x', 'sqft_lot_x', 'dist_city^2',
       'dist_city sqft_living', 'sqft_living^2', 'sqft_living sqft_lot',
       'sqft_lot^2', 'population', 'price', 'bedrooms', 'bathrooms',
       'sqft_living_y', 'sqft_lot_y', 'sqft_above', 'sqft_basement',
       'yr_built', 'yr_renovated', 'lat', 'sqft_living15', 'sqft_lot15',
       'sqft_liv_lot', 'yr_updated', 'dist_city_y', 'zip_98002', 'zip_98003',
       'zip_98004', 'zip_98005', 'zip_98006', 'zip_98008', 'zip_98010',
       'zip_98019', 'zip_98022', 'zip_98023', 'zip_98027', 'zip_98029',
       'zip_98030', 'zip_98031', 'zip_98032', 'zip_98033', 'zip_98038',
       'zip_98039', 'zip_98040', 'zip_98042', 'zip_98045', 'zip_98052',
       'zip_98053', 'zip_98055', 'zip_98056', 'zip_98058', 'zip_98074',
       'zip_98075', 'zip_98077', 'zip_98092', 'zip_98102', 'zip_98105',
       'zip_98106', 'zip_98108', 'zip_98109', 'zip_98112', 'zip_98115',
       'zip_98116', 'zip_98118', 'zip_98119', 'zip_98122', 'zip_98

### 6.2) After removing the features, re-run Step 5 and see if your new model performs better than the old model.

In [99]:
best_features = ['dist_city_x', 'sqft_living_x', 'sqft_lot_x', 'dist_city^2',
       'dist_city sqft_living', 'sqft_living^2', 'sqft_living sqft_lot',
       'sqft_lot^2', 'population', 'bedrooms', 'bathrooms',
       'sqft_living_y', 'sqft_lot_y', 'sqft_above', 'sqft_basement',
       'yr_built', 'yr_renovated', 'lat', 'sqft_living15', 'sqft_lot15',
       'sqft_liv_lot', 'yr_updated', 'dist_city_y', 'zip_98002', 'zip_98003',
       'zip_98004', 'zip_98005', 'zip_98006', 'zip_98008', 'zip_98010',
       'zip_98019', 'zip_98022', 'zip_98023', 'zip_98027', 'zip_98029',
       'zip_98030', 'zip_98031', 'zip_98032', 'zip_98033', 'zip_98038',
       'zip_98039', 'zip_98040', 'zip_98042', 'zip_98045', 'zip_98052',
       'zip_98053', 'zip_98055', 'zip_98056', 'zip_98058', 'zip_98074',
       'zip_98075', 'zip_98077', 'zip_98092', 'zip_98102', 'zip_98105',
       'zip_98106', 'zip_98108', 'zip_98109', 'zip_98112', 'zip_98115',
       'zip_98116', 'zip_98118', 'zip_98119', 'zip_98122', 'zip_98125',
       'zip_98126', 'zip_98133', 'zip_98144', 'zip_98146', 'zip_98148',
       'zip_98155', 'zip_98168', 'zip_98177', 'zip_98178', 'zip_98188',
       'zip_98198', 'zip_98199', 'sch_3', 'sch_5', 'con_2', 'con_4', 'con_5',
       'vie_1', 'vie_2', 'vie_3', 'vie_4', 'flo_2.0', 'flo_2.5', 'flo_3.5',
       'gra_4', 'gra_5', 'gra_6', 'gra_7', 'gra_9', 'gra_10', 'gra_11',
       'gra_12', 'gra_13', 'wat_1']

## Step 7: Evaluate your different models in order to determine the best model overall.

In [102]:
#instantiate a linear regression object
lm_rfe = LinearRegression()
#fit the linear regression to the data
lm_rfe = lm.fit(X_train, y_train)

In [104]:
#your code here 
y_train_pred_rfe = lm_rfe.predict(X_train)
y_test_pred_rfe = lm_rfe.predict(X_test)

In [105]:
train_rmse = np.sqrt(metrics.mean_squared_error(y_train, y_train_pred_rfe))
test_rmse = np.sqrt(metrics.mean_squared_error(y_test, y_test_pred_rfe))
print('Training Root Mean Squared Error:' , train_rmse)
print('Training Root Mean Squared Error:' , test_rmse)

Training Root Mean Squared Error: 142360.20237314573
Training Root Mean Squared Error: 132004.42851708794


In [106]:
scaler1 = StandardScaler()

In [107]:
scaler1.fit(kc_df[summary_features])

StandardScaler()

In [108]:
scaled_data2 = scaler1.transform(kc_df[summary_features])
df_features_scaled = pd.DataFrame(data=scaled_data2, columns=summary_features, index=kc_df.index)

In [109]:
lm_final = LinearRegression()
lim_final = lm.fit(df_features_scaled, target)

In [110]:
df_features_scaled

,population,bedrooms,bathrooms,sqft_living,sqft_lot,sqft_above,sqft_basement,yr_built,yr_renovated,lat,...,gra_5,gra_6,gra_7,gra_8,gra_9,gra_10,gra_11,gra_12,gra_13,wat_1
0,-0.125673,1.784465,0.828598,0.469905,-0.536551,0.834129,-0.805717,1.472531,-0.209349,-2.166476,...,-0.105969,-0.324588,-0.843028,1.602005,-0.371718,-0.233369,-0.137101,-0.064666,-0.025231,-0.089698
1,-0.125673,-0.406516,0.503706,-0.276521,-0.470094,0.049074,-0.805717,1.131730,-0.209349,-1.853633,...,-0.105969,-0.324588,1.186201,-0.624218,-0.371718,-0.233369,-0.137101,-0.064666,-0.025231,-0.089698
2,-0.125673,1.784465,0.178813,0.285305,-0.373900,-0.626242,1.241131,-0.129232,-0.209349,-1.625323,...,-0.105969,-0.324588,1.186201,-0.624218,-0.371718,-0.233369,-0.137101,-0.064666,-0.025231,-0.089698
3,-0.125673,-0.406516,-1.445650,-1.056197,0.521374,-0.770953,-0.805717,-2.139956,-0.209349,-2.055210,...,9.436741,-0.324588,-0.843028,-0.624218,-0.371718,-0.233369,-0.137101,-0.064666,-0.025231,-0.089698
4,-0.125673,-0.406516,-1.445650,-1.296979,-0.058722,-1.024197,-0.805717,-2.105876,-0.209349,-2.050153,...,9.436741,-0.324588,-0.843028,-0.624218,-0.371718,-0.233369,-0.137101,-0.064666,-0.025231,-0.089698
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17285,-0.988677,-1.502007,-1.445650,-1.446035,-0.366571,-1.180967,-0.805717,-0.981234,-0.209349,0.644773,...,-0.105969,3.080829,-0.843028,-0.624218,-0.371718,-0.233369,-0.137101,-0.064666,-0.025231,-0.089698
17286,-0.988677,0.688974,0.503706,1.856123,-0.037710,0.483206,1.241131,-0.674513,-0.209349,0.602146,...,-0.105969,-0.324588,-0.843028,-0.624218,-0.371718,4.285056,-0.137101,-0.064666,-0.025231,-0.089698
17287,-0.988677,-0.406516,0.503706,1.707068,-0.239731,-0.167992,1.241131,-0.776753,-0.209349,0.630323,...,-0.105969,-0.324588,-0.843028,1.602005,-0.371718,-0.233369,-0.137101,-0.064666,-0.025231,-0.089698
17288,-0.988677,-1.502007,0.503706,0.743938,-0.475958,-0.107696,1.241131,-1.185714,-0.209349,0.566021,...,-0.105969,-0.324588,-0.843028,1.602005,-0.371718,-0.233369,-0.137101,-0.064666,-0.025231,-0.089698


In [111]:
y_pred = lm.predict(df_features_scaled)
y_real = kc_df['price']
rmse = np.sqrt(metrics.mean_squared_error(y_real, y_pred))
print('Real: ', int(rmse))

Real:  152217


## Step 8:  Refit your best model to the entire dataset.

## Step 9: Save your final model using pickle.

https://machinelearningmastery.com/save-load-machine-learning-models-python-scikit-learn/

In [61]:
def scale_fit_pickle_origin(df_features, target):
    """
    Scaling df with features,
    Fit linear model with scaled features
    Create pickle file with Scaler and Model
    params: 
            df_features - most important features 
            target - Series
    """
    scaler = StandardScaler()
    # fit the scaler to the training data
    scaler.fit(df_features)
    #transform the training data
    scaled_data = scaler.transform(df_features)
    #create dataframe
    df_features_scaled = pd.DataFrame(data=scaled_data, columns=df_features.columns, index=df_features.index)
    lm_final = LinearRegression()
    #fit the linear regression to the data
    lm_final = lm_final.fit(df_features_scaled, target)
    pickle_out = open("model.pickle","wb")
    pickle.dump(lm_final, pickle_out)
    pickle_out.close()
    pickle_out = open('scaler.pickle', "wb")
    pickle.dump(scaler, pickle_out)
    pickle_out.close()
    return print(' CONGRATS !!! You sucessfuly created you pickles for SCALER and MODEL')

In [63]:
scale_fit_pickle_origin(kc_df_poly2[summary_features1], target)

 CONGRATS !!! You sucessfuly created you pickles for SCALER and MODEL


In [122]:
kc_df_poly2.columns

Index(['dist_city_x', 'sqft_living_x', 'sqft_lot_x', 'dist_city^2',
       'dist_city sqft_living', 'dist_city sqft_lot', 'sqft_living^2',
       'sqft_living sqft_lot', 'sqft_lot^2', 'population',
       ...
       'gra_5', 'gra_6', 'gra_7', 'gra_8', 'gra_9', 'gra_10', 'gra_11',
       'gra_12', 'gra_13', 'wat_1'],
      dtype='object', length=122)